In [34]:
%%writefile src/OrderEvent.sol
pragma solidity 0.5.1;
contract OrderEvent {
    address private owner;
    constructor() public {
        owner = msg.sender;
    }
    event OrderLog(address indexed from, bytes2 itemId, uint quantity, string dest);
    function order(bytes2 _itemId, uint _quantity, string memory _dest) public payable {
        emit OrderLog(msg.sender, _itemId, _quantity, _dest);
    }
     modifier isOwner() {
        if(msg.sender != owner) {
            revert();
        }
        _;
    }
    function getBalance() view public isOwner returns(uint) {
        return address(this).balance;
    }
}

Overwriting src/OrderEvent.sol


In [35]:
!solc --abi --bin --gas src/OrderEvent.sol


======= src/OrderEvent.sol:OrderEvent =======
Gas estimation:
construction:
   20468 + 158000 = 178468
external:
   getBalance():	877
   order(bytes2,uint256,string):	infinite
Binary: 
608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550610316806100606000396000f3fe608060405260043610610046576000357c01000000000000000000000000000000000000000000000000000000009004806312065fe01461004b5780637a59a92914610076575b600080fd5b34801561005757600080fd5b50610060610166565b6040518082815260200191505060405180910390f35b6101646004803603606081101561008c57600080fd5b8101908080357dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff1916906020019092919080359060200190929190803590602001906401000000008111156100de57600080fd5b8201836020820111156100f057600080fd5b8035906020019184600183028401116401000000008311171561011257600080fd5b91908080601f016020809104026020016040519081016040528093929190818152602001

In [36]:
%%writefile src/OrderEventDeploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var _abiArray  =[{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_itemId","type":"bytes2"},{"name":"_quantity","type":"uint256"},{"name":"_dest","type":"string"}],"name":"order","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"name":"from","type":"address"},{"indexed":false,"name":"itemId","type":"bytes2"},{"indexed":false,"name":"quantity","type":"uint256"},{"indexed":false,"name":"dest","type":"string"}],"name":"OrderLog","type":"event"}];
var _bin = "0x" + "608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550610316806100606000396000f3fe608060405260043610610046576000357c01000000000000000000000000000000000000000000000000000000009004806312065fe01461004b5780637a59a92914610076575b600080fd5b34801561005757600080fd5b50610060610166565b6040518082815260200191505060405180910390f35b6101646004803603606081101561008c57600080fd5b8101908080357dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff1916906020019092919080359060200190929190803590602001906401000000008111156100de57600080fd5b8201836020820111156100f057600080fd5b8035906020019184600183028401116401000000008311171561011257600080fd5b91908080601f016020809104026020016040519081016040528093929190818152602001838380828437600081840152601f19601f8201169050808301925050505050505091929192905050506101e0565b005b60008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff161415156101c357600080fd5b3073ffffffffffffffffffffffffffffffffffffffff1631905090565b3373ffffffffffffffffffffffffffffffffffffffff167f0cee52e22e6b387c4183ee502522fa33f030e118cc6065c7a842e8b7c00e4a6a84848460405180847dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff19167dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff1916815260200183815260200180602001828103825283818151815260200191508051906020019080838360005b838110156102a957808201518184015260208101905061028e565b50505050905090810190601f1680156102d65780820380516001836020036101000a031916815260200191505b5094505050505060405180910390a250505056fea165627a7a72305820a32f94ff492f19a0fa633bef4435844a70236659de3f23df50381cf4885718070029";

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 364124}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/OrderEventDeploy.js


In [37]:
!node src/OrderEventDeploy.js

Deploying the contract from 0x8cfB179861692D9D630a4E7EbCbdbCf609247d69
hash: 0x5cc17df9e44affa8bfa932555c307e7970191c30666ebb9919ef0338576c4d75
---> The contract deployed to: 0xf95A0E974359A5711dcb5B12B0F809bCe9a614F7


In [38]:
%%writefile src/OrderEventUse.js
var Web3=require('web3');
var fs=require('fs');
var web3 = new Web3(new Web3.providers.WebsocketProvider("http://localhost:8345"));
var _abiArray=[{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_itemId","type":"bytes2"},{"name":"_quantity","type":"uint256"},{"name":"_dest","type":"string"}],"name":"order","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"name":"from","type":"address"},{"indexed":false,"name":"itemId","type":"bytes2"},{"indexed":false,"name":"quantity","type":"uint256"},{"indexed":false,"name":"dest","type":"string"}],"name":"OrderLog","type":"event"}];

async function doIt() {
    var _order = new web3.eth.Contract(_abiArray, '0xf95A0E974359A5711dcb5B12B0F809bCe9a614F7');
    var event = _order.events.OrderLog({fromBlock: 0}, function (error, result) {
        if (!error) {
            log = JSON.stringify(result.returnValues);
            console.log("Event fired: " + log);
            fs.appendFile("src/OrderEvent.txt", log, "utf-8", function(e) {
                if(!e) {
                    console.log(">> Writing to file");
                }
            });
        }
    });
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    value = await _order.methods.order("0x1111", 3, "20 2-gil Hongji-dong Jongro-gu Seoul").send({from: accounts[0], gas: 100000});
    console.log("---> order called " + JSON.stringify(value.events.OrderLog.returnValues));
    value = await _order.methods.order("0x1111", 5, "20 2-gil Hongji-dong Jongro-gu Seoul").send({from: accounts[0], gas: 100000});
    console.log("---> order called " + JSON.stringify(value.events.OrderLog.returnValues));
    value = await _order.methods.order("0x1111", 20, "20 2-gil Hongji-dong Jongro-gu Seoul").send({from: accounts[0], gas: 100000});
    console.log("---> order called " + JSON.stringify(value.events.OrderLog.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}

doIt()

Overwriting src/OrderEventUse.js


In [39]:
!node src/OrderEventUse.js

Account: 0x8cfB179861692D9D630a4E7EbCbdbCf609247d69
Balance before: 99995091200000000000
Event fired: {"0":"0x8cfB179861692D9D630a4E7EbCbdbCf609247d69","1":"0x1111","2":"3","3":"20 2-gil Hongji-dong Jongro-gu Seoul","from":"0x8cfB179861692D9D630a4E7EbCbdbCf609247d69","itemId":"0x1111","quantity":"3","dest":"20 2-gil Hongji-dong Jongro-gu Seoul"}
>> Writing to file
---> order called {"0":"0x8cfB179861692D9D630a4E7EbCbdbCf609247d69","1":"0x1111","2":"3","3":"20 2-gil Hongji-dong Jongro-gu Seoul","from":"0x8cfB179861692D9D630a4E7EbCbdbCf609247d69","itemId":"0x1111","quantity":"3","dest":"20 2-gil Hongji-dong Jongro-gu Seoul"}
Event fired: {"0":"0x8cfB179861692D9D630a4E7EbCbdbCf609247d69","1":"0x1111","2":"5","3":"20 2-gil Hongji-dong Jongro-gu Seoul","from":"0x8cfB179861692D9D630a4E7EbCbdbCf609247d69","itemId":"0x1111","quantity":"5","dest":"20 2-gil Hongji-dong Jongro-gu Seoul"}
>> Writing to file
---> order called {"0":"0x8cfB179861692D9D630a4E7EbCbdbCf609247d69","1":"0x1111","2":"5","3

In [40]:
!type src\OrderEvent.txt

{"0":"0x8cfB179861692D9D630a4E7EbCbdbCf609247d69","1":"0x1111","2":"3","3":"20 2-gil Hongji-dong Jongro-gu Seoul","from":"0x8cfB179861692D9D630a4E7EbCbdbCf609247d69","itemId":"0x1111","quantity":"3","dest":"20 2-gil Hongji-dong Jongro-gu Seoul"}{"0":"0x8cfB179861692D9D630a4E7EbCbdbCf609247d69","1":"0x1111","2":"5","3":"20 2-gil Hongji-dong Jongro-gu Seoul","from":"0x8cfB179861692D9D630a4E7EbCbdbCf609247d69","itemId":"0x1111","quantity":"5","dest":"20 2-gil Hongji-dong Jongro-gu Seoul"}{"0":"0x8cfB179861692D9D630a4E7EbCbdbCf609247d69","1":"0x1111","2":"20","3":"20 2-gil Hongji-dong Jongro-gu Seoul","from":"0x8cfB179861692D9D630a4E7EbCbdbCf609247d69","itemId":"0x1111","quantity":"20","dest":"20 2-gil Hongji-dong Jongro-gu Seoul"}
